In [1]:
import pandas as pd
import numpy as np
import os, sys, re

In [2]:
sys.path.append("./static/python/")

In [120]:
import userinput

In [311]:
reload(userinput)

<module 'userinput' from './static/python/userinput.pyc'>

In [312]:
fn = r"/Users/dblyon/CloudStation/CPR/BTW_GO/UserInput.txt"
fn = r"/Users/dblyon/modules/cpr/agotool/static/data/exampledata/exampledata_yeast.txt"
fn = r"/Users/dblyon/modules/cpr/agotool/static/data/exampledata/refactorUserinput.txt"
df = pd.read_csv(fn, sep='\t', decimal=",")#, converters={"background_int": float, "background_an": str, "foreground_an": str})

In [313]:
# fh = open(fn, "r").read()
ui = userinput.Userinput(user_input_fn=fn, foreground_string=None, background_string=None,
col_foreground='foreground', col_background='background', col_intensity='intensity',
num_bins=100, decimal='.', method="abundance_correction")

In [314]:
cond = ui.foreground["intensity"] > -1
bins = pd.cut(ui.foreground.loc[cond, "intensity"], bins=100, retbins=True)[1]
bins = [-1] + list(bins)

In [315]:
ui.foreground

,foreground,intensity
0,P00549,1.700000
1,AN3,1.700000
2,P00330,2.600000
3,P05743,3.642069
4,P02407,4.640611
5,P0C2H7,5.636197
6,P35997;P38711,7.623177
7,YOYOYO,-1.000000


In [316]:
for x in ui.iter_bins():
#     print wf, bins
#     print bg, fg
#     print ans, cf
    print x
    print

(['YOYOYO'], ['YOYOYO'], 1, '(-2, 1.694]')

(['AN2;P00549', 'AN3'], ['P00549', 'AN3'], 1.0, '(1.694, 1.759]')

(['P00330'], ['P00330'], 1.0, '(2.588, 2.648]')

(['P05743'], ['P05743'], 1.0, '(3.595, 3.655]')

(['P02407;P14127'], ['P02407'], 1.0, '(4.602, 4.662]')

(['P0C2H6;P0C2H7'], ['P0C2H7'], 1.0, '(5.609, 5.669]')

(['P35997;P38711', 'AN3000'], ['P35997;P38711'], 0.5, '(7.564, 7.623]')



In [323]:
mbps = 30.0
gb = mbps / 1000.0
min_ = gb * 60
hour = min_ * 60
print min_
print hour
tb = 2.0
(tb * 1000) / hour

1.8
108.0


18.518518518518523

In [157]:
bins = pd.cut(ui.foreground["intensity"], bins=100, retbins=True)[1]
groups_fg = ui.foreground.groupby(pd.cut(ui.foreground["intensity"], bins=bins))
groups_bg = ui.background.groupby(pd.cut(ui.background["intensity"], bins=bins))

In [160]:
# ui.background[ui.background["intensity"] <= 0]

In [159]:
for group_fg, group_bg in zip(groups_fg, groups_bg):
    bins_in_group = group[0]
    pg_fg = group_fg[1]["foreground"]
    pg_bg = group_bg[1]["background"]
    print pg_fg, len(pg_fg)
    print pg_bg, len(pg_bg)
    print 
    break

0       BUBU;BUBU2;BUBU3
1       AUBU;AUBU2;AUBU3
3                 P09201
4                 P14066
6                 P20050
9                 P38142
11                P39959
16                Q03868
17                Q04004
18                Q04383
20                Q06592
23                Q12348
110               Q2V2Q1
112               Q08981
145               P51979
212               P38806
411               P07261
446               Q05471
483               P40963
574               P38157
702               P32833
787               P38282
806               P50104
981               Q12247
992               P40316
1029              P31111
1046              P38140
1055              P53718
1079              P32389
1109              Q12472
1119              Q9URQ3
1123              P18412
Name: foreground, dtype: object 32
0    BUBU
Name: background, dtype: object 1



In [116]:
# for group in groups:
# #     bins_in_group = group[0]
# #     pg_in_group = group[1]["foreground"]
# #     print bins_in_group
# #     print pg_in_group
#     print group
#     print 

In [80]:
hist, bins = np.histogram(ui.foreground["intensity"], bins=100)

In [82]:
bins # (-1.0121, -0.879]

array([ -1.        ,  -0.87922487,  -0.75844974,  -0.6376746 ,
        -0.51689947,  -0.39612434,  -0.2753492 ,  -0.15457407,
        -0.03379894,   0.08697619,   0.20775133,   0.32852646,
         0.44930159,   0.57007672,   0.69085186,   0.81162699,
         0.93240212,   1.05317725,   1.17395239,   1.29472752,
         1.41550265,   1.53627778,   1.65705291,   1.77782805,
         1.89860318,   2.01937831,   2.14015345,   2.26092858,
         2.38170371,   2.50247884,   2.62325398,   2.74402911,
         2.86480424,   2.98557937,   3.10635451,   3.22712964,
         3.34790477,   3.4686799 ,   3.58945504,   3.71023017,
         3.8310053 ,   3.95178043,   4.07255557,   4.1933307 ,
         4.31410583,   4.43488096,   4.5556561 ,   4.67643123,
         4.79720636,   4.91798149,   5.03875663,   5.15953176,
         5.28030689,   5.40108202,   5.52185716,   5.64263229,
         5.76340742,   5.88418255,   6.00495769,   6.12573282,
         6.24650795,   6.36728308,   6.48805822,   6.60

In [85]:
hist

array([30,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,
        0,  1,  0,  0,  1,  1,  1,  3,  0,  5,  5,  6,  7, 12, 14, 19, 15,
       17, 19, 25, 28, 21, 36, 33, 44, 47, 61, 57, 59, 48, 42, 51, 63, 46,
       45, 54, 46, 40, 28, 18, 18, 21, 15, 26, 16,  7,  4,  0,  2])

In [83]:
10.71518785 -  10.83596299

-0.120775140000001

In [84]:
9.62821166 -   9.74898679

-0.12077513000000017

In [ ]:
df.loc[0, "background_int"]

In [ ]:
# np.histogram(df["background_int"], bins=10)
np.histogram(df.loc[pd.notnull(df["background_int"]), "background_int"], bins=10)

In [ ]:
ui.background_df

In [ ]:
# import math

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

In [ ]:
# arr = list(np.arange(10))
# arr += [2.3]*8
# hist, bin_edges = np.histogram(arr, bins=10, density=False)
# plt.hist(arr, bins=bin_edges)  # plt.hist passes it's arguments to np.histogram
# plt.show()

In [ ]:
# good stuff

In [ ]:
# foreground proteinGroups with intensities
intensity_foreground = []
for proteinGroup in ui.foreground_ser:
    an_first_in_proteinGroup = proteinGroup.split(";")[0]
    intensity_foreground.append(ui.an_2_intensity_dict[an_first_in_proteinGroup])
#     try:      
#         intensity_foreground.append(ui.an_2_intensity_dict[an_first_in_proteinGroup])
#     except KeyError:
#         # can't occur, since defaultdict
#         raise StopIteration

In [ ]:
ui.foreground_df = pd.DataFrame(data=intensity_foreground, columns=["Int"])
ui.foreground_df["AN"] = ui.foreground_ser

In [ ]:
df = ui.foreground_df
df

In [ ]:
1.5*30

In [ ]:
# create bins
# which proteinGroups from background are within bin

In [ ]:
num_bins = 100
bins = np.linspace(df["Int"].min(), df["Int"].max(), num_bins)
groups = df.groupby(pd.cut(df["Int"], bins))

In [ ]:
# for group in groups:
#     print group
#     print